# Поиск гиперпараметров для модели поиска каверов

На основе подготовленного в основном ноутбуке датасета проведём поиск гиперпараметров для моделии классификации CatBoost.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, recall_score
from optuna.samplers import TPESampler
import optuna
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

Откроем подготовленный датасет

In [2]:
data = pd.read_csv('datasets/meta_for_training.csv', sep='\t', encoding='utf-8')

In [3]:
data = data.drop('Unnamed: 0', axis=1)

Разделим датасет на тренировочную и тестовую выборку

In [4]:
train_data, data_test = train_test_split(data, stratify=data['target'], test_size=0.2)


## Обучение модели

Создадим trial-объект для поиска гиперпараметров с помощью optuna.

In [5]:
def objective(trial):
    # выделим из тестовой выборки валидационную
    data_train, data_valid = train_test_split(train_data, stratify=train_data['target'], test_size=0.2)
    # разделим параметры и признаки в обучающей
    features_train = data_train.drop('target', axis=1)
    target_train = data_train['target']
    # проведём ресэмплирование обучающей выборки
    resample = SMOTETomek(tomek=TomekLinks(sampling_strategy='auto'))
    features_samp, target_samp = resample.fit_resample(features_train, target_train)
    # разделим параметры и признаки в валидационной выборке
    features_valid = data_valid.drop('target', axis=1)
    target_valid = data_valid['target']  
    
    # зададим сетку гиперпараметров модели
    model = CatBoostClassifier(task_type="GPU",
        iterations=trial.suggest_int("iterations", 1000, 2000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=False
    )
    # зададим дополнительный аттрибут для сохранения лучшей модели
    trial.set_user_attr(key="best_booster", value=model)
    # обучим модель
    model.fit(features_samp, target_samp)
    # получим прогнозы для валидационной выборки
    y_pred = model.predict(features_valid)
    # получим метрику модели
    score = recall_score(target_valid, y_pred)

    return score

Зададим дополнителный колбэк для сохранения модели с лучшей метрикой.

In [6]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_booster", value=trial.user_attrs["best_booster"])

Проведём испытаний моделей с различными параметрами.

In [7]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

cat_study = optuna.create_study(study_name="catboost", direction="maximize", sampler=TPESampler())
cat_study.optimize(objective, n_trials=100, callbacks=[callback],
                   #timeout=3600
                   )
cat_model = cat_study.user_attrs["best_booster"]

Выведеи информацию о лучшей модели.

In [8]:
print("Number of finished trials: ", len(cat_study.trials))
print("Best trial:")
cat_trial = cat_study.best_trial
print("  Value: ", cat_trial.value)
print("  Params: ")
for key, value in cat_trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  100
Best trial:
  Value:  1.0
  Params: 
    iterations: 1403
    learning_rate: 0.5663239666177774
    depth: 9
    l2_leaf_reg: 3.829566095968917e-05
    bootstrap_type: Bayesian
    random_strength: 1.877689956142966e-06
    bagging_temperature: 9.08772716092541
    od_type: Iter
    od_wait: 10


Посмотри на результатыт проведённых испытаний.

In [9]:
optuna.visualization.plot_optimization_history(cat_study)

Модель ...

Посмотрим на степень влияния подобранных гиперпараметров на качество модели. 

In [10]:
optuna.visualization.plot_param_importances(cat_study)

Все гиперпараметры имеют достаточное влияние на итоговый результат (у 'bootstrap_type' одно категориальное значение).

Выделим параметры и признаки на тестовой выборке.

In [11]:
features_test = data_test.drop('target', axis=1)
target_test = data_test['target']

## Вывод по результатам обучения модели

Оценим качество модели на тестовой выборке

In [12]:
print(classification_report(target_test, cat_model.predict(data=features_test), digits=5))

              precision    recall  f1-score   support

           0    0.99912   0.99384   0.99647      1137
           1    0.96175   0.99435   0.97778       177

    accuracy                        0.99391      1314
   macro avg    0.98043   0.99410   0.98713      1314
weighted avg    0.99408   0.99391   0.99395      1314



На тестовой выборке модель обнаружила 99,5% схожих треков, что достаточно хорошо, но при получении большого количества прогнозов (нескольько тысяч) будет наблюдаться некоторое количество ошибок.

Сохраним модель

In [13]:
cat_model.save_model("models/cvr_clsfr_mdl_tuned_100.cbm")